##### Grading Feedback Cell
75

# IST 718: Big Data Analytics

- Professor: Willard Williamson <wewillia@syr.edu>
- Faculty Assistant: Palaniappan Muthukkaruppan
## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers from your classmates.  Short code snippets are allowed from the internet.  Any code is allowed from the class text books or class provided code.__
- Please do not change the file names. The FAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and FAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`).

In [1]:
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
from pyspark.sql import Row
import numpy as np
import re

from pyspark.sql.types import StructType,StructField

import types


import numpy.testing as testing

spark = SparkSession.builder.appName('map-reduce').getOrCreate()
sc = spark.sparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Warning: Use exclusively Spark 1.6 when asked to do so and Spark 2.0 (dataframes) only in the last question. Do not use Pandas at all in this assignment

# Part 2: Data cleaning and basic analyses

In this part, you will learn to read data from non-standard formats, clean data, and produce some basic analysis of it.

We will use Spark 1.6 (`sparkContext` on variable `sc`) to load text files from which we will extract features that are predictive of a target value. Unfortunately, the data is stored in some non-standard format where each line contains the customer index, the feature index, and the value of the feature for that customer. Similarly, the target files contain in each line the customer index and the target value. We will load these files into two RDDs:

In [2]:
# Adjust the following lines as needed to read the data files
# If running on databricks, you will need to upload the data to databricks and then
# adjust the file path as demonstrated in class.
raw_features_rdd = sc.textFile('hw2_dataset_features.txt')
raw_target_rdd = sc.textFile('hw2_dataset_targets.txt')

An issue with the data is that there were problems transmitting the features and targets. If this happened, then a text `ERROR` or `ERROR TRANSFERRING` replaced the data. If you look at the first 10 values of the features and target RDD, you will see these types of lines:

```python
raw_features_rdd.take(10)
```

```
['<customer_feature customer_id=0 feature_id=0>-0.57</customer_feature>',
 '<customer_feature customer_id=0 feature_id=1>-0.38</customer_feature>',
 '<customer_feature customer_id=0 feature_id=2>0.00</customer_feature>',
 '<customer_feature customer_id=0 feature_id=3>-0.07</customer_feature>',
 '<customer_feature customer_id=0 feature_id=4>-0.28</customer_feature>',
 '<customer_feature customer_id=0 feature_id=5>-0.79</customer_feature>',
 'ERROR',
 '<customer_feature customer_id=0 feature_id=7>0.28</customer_feature>',
 '<customer_feature customer_id=0 feature_id=8>1.65</customer_feature>',
 '<customer_feature customer_id=0 feature_id=9>0.57</customer_feature>']
```

```python
raw_target_rdd.take(35)
```

```
['<customer_target customer_id=0>-252.49</customer_target>',
 '<customer_target customer_id=1>36.67</customer_target>',
 '<customer_target customer_id=2>138.02</customer_target>',
 '<customer_target customer_id=3>-429.54</customer_target>',
 '<customer_target customer_id=4>-18.23</customer_target>',
 '<customer_target customer_id=5>-5.52</customer_target>',
 '<customer_target customer_id=6>-31.96</customer_target>',
 'ERROR TRANSFERRING',
 'ERROR TRANSFERRING',
 '<customer_target customer_id=9>-111.88</customer_target>']
```

In [3]:
# try it yourself: raw_features_rdd.take(10) and raw_target_rdd.take(10)
raw_features_rdd.take(10) 

['<customer_feature customer_id=0 feature_id=0>-0.57</customer_feature>',
 '<customer_feature customer_id=0 feature_id=1>-0.38</customer_feature>',
 '<customer_feature customer_id=0 feature_id=2>0.00</customer_feature>',
 '<customer_feature customer_id=0 feature_id=3>-0.07</customer_feature>',
 '<customer_feature customer_id=0 feature_id=4>-0.28</customer_feature>',
 '<customer_feature customer_id=0 feature_id=5>-0.79</customer_feature>',
 'ERROR',
 '<customer_feature customer_id=0 feature_id=7>0.28</customer_feature>',
 '<customer_feature customer_id=0 feature_id=8>1.65</customer_feature>',
 '<customer_feature customer_id=0 feature_id=9>0.57</customer_feature>']

In [4]:
raw_target_rdd.take(10)

['<customer_target customer_id=0>-252.49</customer_target>',
 '<customer_target customer_id=1>36.67</customer_target>',
 '<customer_target customer_id=2>138.02</customer_target>',
 '<customer_target customer_id=3>-429.54</customer_target>',
 '<customer_target customer_id=4>-18.23</customer_target>',
 '<customer_target customer_id=5>-5.52</customer_target>',
 '<customer_target customer_id=6>-31.96</customer_target>',
 'ERROR TRANSFERRING',
 'ERROR TRANSFERRING',
 '<customer_target customer_id=9>-111.88</customer_target>']

## Question 2.1 (10 pts):

Filter out the lines that contain errors and store them in `raw_features2_rdd` and `raw_targets_rdd` respectively. 

In [5]:
def filter_err1(row):
    
    if row == 'ERROR':
        return False
    else:
        return True


In [6]:
raw_features2_rdd= raw_features_rdd.filter(lambda line: filter_err1(line))
raw_features2_rdd.count()

95036

In [7]:
# create raw_features2_rdd and raw_targets2_rdd below
# YOUR CODE HERE
def filter_err2(row):
    
    if row == 'ERROR TRANSFERRING':
        return False
    else:
        return True


In [8]:
raw_target2_rdd= raw_target_rdd.filter(lambda line: filter_err2(line))
raw_target2_rdd.count()

8968

In [9]:
# check that things work
print(raw_features2_rdd.count())
print(raw_target2_rdd.count())

95036
8968


In [10]:
"""10 pts: Check that the lines are properly discarded"""
assert raw_features2_rdd.count() == 95036
assert raw_target2_rdd.count() == 8968

## Question 2.2 (10 pts):
You will further process `raw_features2_rdd` such that you will create a key-value RDD of the following form: the key is the customer index as an integer and the value is a dictionary whose key is a string `f_0`, `f_1`, ..., `f_9` for feature index 0, 1, ... 9, respetively, and the value is a floating point number of the feature value. 

Define a function `map_features2` that performs such key-value pair creation.

In [11]:

def map_features2(line):
    match = re.search('.*customer_id=(\d{1,}).*feature_id=(\d{1})>(-?\d*[.,]?\d{1,})',line)
    return [int(match.group(1)),{'f_'+str(match.group(2)):float(match.group(3))}]
    

For example, for the input element:

`'<customer_feature customer_id=4 feature_id=0>-0.79</customer_feature>'`

it should generate
```python
[4, {'f_0': -0.79}]
```

In [12]:
# test it here
raw_features2_rdd.\
    map(map_features2).\
    take(10)

[[0, {'f_0': -0.57}],
 [0, {'f_1': -0.38}],
 [0, {'f_2': 0.0}],
 [0, {'f_3': -0.07}],
 [0, {'f_4': -0.28}],
 [0, {'f_5': -0.79}],
 [0, {'f_7': 0.28}],
 [0, {'f_8': 1.65}],
 [0, {'f_9': 0.57}],
 [1, {'f_0': 0.5}]]

In [13]:
"""10 pts: Check that the new raw_features2_rdd and raw_target2_rdd RDDs are correct"""
# key is an integer
np.testing.assert_equal(type(raw_features2_rdd.map(map_features2).first()[0]), int)
# value is a dictionary
np.testing.assert_equal(type(raw_features2_rdd.map(map_features2).first()[1]), dict)

## Question 2.3 (5 pts):

You will create a function `map_target2` that will be applied to `raw_target2_rdd`. This function will create key-value pair where the key is the customer index as an integer and the value is the floating point representation of the target. Assign the resulting RDD to `raw_target3_rdd`. 

In [14]:
def map_target2(line):
    match1 = re.search('.*customer_id=(\d{1,})>(-?\d*[.,]?\d{1,})',line)
    return int(match1.group(1)),float(match1.group(2))
    raise NotImplementedError()

# make the assignment here
raw_target3_rdd = raw_target2_rdd.map(map_target2)
raw_target3_rdd.collect()



[(0, -252.49),
 (1, 36.67),
 (2, 138.02),
 (3, -429.54),
 (4, -18.23),
 (5, -5.52),
 (6, -31.96),
 (9, -111.88),
 (10, -34.48),
 (11, -172.76),
 (12, -356.98),
 (13, -77.59),
 (14, -106.22),
 (16, 117.49),
 (18, -142.74),
 (19, 176.36),
 (20, 257.51),
 (21, -136.39),
 (22, -117.16),
 (23, 16.39),
 (24, 21.24),
 (25, -158.04),
 (26, 187.7),
 (27, -29.31),
 (28, -234.14),
 (30, -103.26),
 (31, -30.04),
 (32, 116.33),
 (33, 204.42),
 (34, -405.09),
 (35, 136.89),
 (36, 49.22),
 (37, -177.24),
 (38, 245.48),
 (39, -98.3),
 (40, -157.39),
 (41, -122.48),
 (42, -35.32),
 (43, -117.87),
 (44, 112.2),
 (45, -261.49),
 (46, -395.52),
 (47, 212.63),
 (48, -62.85),
 (49, -41.15),
 (52, 219.52),
 (53, 43.75),
 (54, 217.91),
 (55, -122.51),
 (57, 235.14),
 (58, 113.28),
 (59, 95.07),
 (60, 33.89),
 (61, -69.14),
 (62, 260.06),
 (63, 53.46),
 (64, -69.14),
 (65, -274.55),
 (66, -183.64),
 (67, 19.01),
 (68, 64.9),
 (69, -179.64),
 (70, 52.69),
 (71, -215.71),
 (72, 125.61),
 (74, -147.76),
 (75, 35.

In [15]:
# make the assignment here
raw_target3_rdd = raw_target2_rdd.map(map_target2)
raw_target3_rdd.take(10)

[(0, -252.49),
 (1, 36.67),
 (2, 138.02),
 (3, -429.54),
 (4, -18.23),
 (5, -5.52),
 (6, -31.96),
 (9, -111.88),
 (10, -34.48),
 (11, -172.76)]

A sample of results:

```python
raw_target2_rdd.map(map_target2).sortByKey().take(5)
```

```
[(0, -252.49), (1, 36.67), (2, 138.02), (3, -429.54), (4, -18.23)]
```

In [16]:
# try it yourself
raw_target2_rdd.map(map_target2).sortByKey().take(5)

[(0, -252.49), (1, 36.67), (2, 138.02), (3, -429.54), (4, -18.23)]

In [17]:
"""5 pts: Check that raw_target3_rdd contains the right values"""
# check types
np.testing.assert_equal(type(raw_target3_rdd.keys().first()), int)
np.testing.assert_equal(type(raw_target3_rdd.values().first()), float)
# the sum of all targets
np.testing.assert_approx_equal(raw_target3_rdd.values().sum(), -179351.71, significant=1)

## Question 2.4 (10 pts):

In this question, you will use map reduce to produce an RDD of key-value pairs where the key is the customer index and the value is a dictionairy with all the features and values associated with that customer. Notice that the map part of the map-reduce is already defined by `map_features2` on `raw_features2_rdd`. Therefore, define the proper `reduce_features2` function to produce the desired results. Create a RDD named `raw_features3_rdd` with the results:

In [18]:
def reduce_features2(v1, v2):
    # YOUR CODE HERE
    return {**v1,**v2}
    raise NotImplementedError()

# Apply mapreduce to produce the raw_features3_rdd from raw_features2_rdd
# YOUR CODE HERE
raw_features3_rdd= raw_features2_rdd.map(map_features2).reduceByKey(reduce_features2)


Running the map reduce should produce the following example result:
```python
raw_features3_rdd.sortByKey().take(2)
```

```console
[(0,
  {'f_0': -0.57,
   'f_1': -0.38,
   'f_2': 0.0,
   'f_3': -0.07,
   'f_4': -0.28,
   'f_5': -0.79,
   'f_7': 0.28,
   'f_8': 1.65,
   'f_9': 0.57}),
 (1,
  {'f_0': 0.5,
   'f_1': 0.8,
   'f_2': -0.49,
   'f_3': 0.25,
   'f_4': 0.37,
   'f_5': 0.73,
   'f_6': -0.43,
   'f_7': 0.89,
   'f_8': -1.85,
   'f_9': -0.44})]
```

In [19]:
raw_features3_rdd.take(3)

[(0,
  {'f_0': -0.57,
   'f_1': -0.38,
   'f_2': 0.0,
   'f_3': -0.07,
   'f_4': -0.28,
   'f_5': -0.79,
   'f_7': 0.28,
   'f_8': 1.65,
   'f_9': 0.57}),
 (2,
  {'f_0': 0.18,
   'f_1': 0.23,
   'f_2': -0.15,
   'f_3': -0.17,
   'f_5': 0.89,
   'f_6': 0.39,
   'f_7': -0.58,
   'f_8': 0.33,
   'f_9': -0.57}),
 (4,
  {'f_0': -0.79,
   'f_1': -0.28,
   'f_2': -0.26,
   'f_3': 0.28,
   'f_4': -0.17,
   'f_5': -0.51,
   'f_6': 0.44,
   'f_7': -0.62,
   'f_8': 1.82,
   'f_9': -0.35})]

In [20]:
"""10 pts: Check that raw_features3_rdd has the correct format and values. There could be hidden tests!"""
# key is an integer
np.testing.assert_equal(type(raw_features3_rdd.first()[0]), int)
# value is a dictionary
np.testing.assert_equal(type(raw_features3_rdd.first()[1]), dict)

## Question 2.5 (20 pts):

Join the two RDDs `raw_target3_rdd` and `raw_features3_rdd` to create an RDD with elements of the form

`[customer_index, (target, feature_dict)]`

where `target` comes from `raw_target3_rdd`, and `feature_dict` is the dictionary with features from `raw_features3_rdd`.

Assign to `rdd_complete` a filtered version of the join for customers who have all 10 features and a target.

Finally use `rdd_complete_rows` to create an RDD of `Row` objects where you map each entry to the following format:

`Row(customer_index,  f_0,  f_1,  f_2,  f_3,  f_4,  f_5,  f_6,  f_7,  f_8,  f_9, target)`

For example, from `rdd_complete`:

```python
rdd_complete.sortByKey().first()
```
should return
```
(1,
 (36.67,
  {'f_0': 0.5,
   'f_1': 0.8,
   'f_2': -0.49,
   'f_3': 0.25,
   'f_4': 0.37,
   'f_5': 0.73,
   'f_6': -0.43,
   'f_7': 0.89,
   'f_8': -1.85,
   'f_9': -0.44}))
```

In [21]:
# create rdd_complete here
# YOUR CODE HERE
rdd_complete=raw_target3_rdd.join(raw_features3_rdd).filter(lambda e:len(e[1][1])==10 and e[1][0] is not None)
#raise NotImplementedError()

In [22]:
rdd_complete.sortByKey().first()

(1,
 (36.67,
  {'f_0': 0.5,
   'f_1': 0.8,
   'f_2': -0.49,
   'f_3': 0.25,
   'f_4': 0.37,
   'f_5': 0.73,
   'f_6': -0.43,
   'f_7': 0.89,
   'f_8': -1.85,
   'f_9': -0.44}))

In [23]:
"""10 pts: Test if `rdd1` has the right data. Remember that there could be hidden tests!"""
# number of elements expected
np.testing.assert_equal(rdd_complete.count(), 5379)

In [24]:
# here define the function to_row that a Row object for an entry of rdd_complete
def to_row(e):
    a = Row(customer_index=e[0])
    a = a.asDict()
    a.update(e[1][1])
    a['target']=e[1][0]
    return Row(**a)
    raise NotImplementedError()

complete_df = rdd_complete.map(to_row).toDF()

In [25]:
# test
rdd_complete.map(to_row).toDF().orderBy('customer_index').show(5)

+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|customer_index|  f_0|  f_1|  f_2|  f_3|  f_4|  f_5|  f_6|  f_7|  f_8|  f_9| target|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|             1|  0.5|  0.8|-0.49| 0.25| 0.37| 0.73|-0.43| 0.89|-1.85|-0.44|  36.67|
|             3| 0.14|-0.87|-0.94| 0.09|-0.69|-0.29|-0.45| -0.6|-1.28|-0.38|-429.54|
|             4|-0.79|-0.28|-0.26| 0.28|-0.17|-0.51| 0.44|-0.62| 1.82|-0.35| -18.23|
|             5|-0.11| 0.86| -1.3|-0.09|-0.12|-0.47| 0.05| 0.98|-1.59|  0.3|  -5.52|
|             9| 0.57| 0.07|-0.31|-0.92|-0.26| -0.9|-0.06|-0.76| 1.39| 0.01|-111.88|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
only showing top 5 rows



In [26]:
# 10 pts test that the creation of the row is successful
assert rdd_complete.map(to_row).toDF()
np.testing.assert_array_equal(rdd_complete.map(to_row).toDF().columns, 
              ['customer_index',
 'f_0',
 'f_1',
 'f_2',
 'f_3',
 'f_4',
 'f_5',
 'f_6',
 'f_7',
 'f_8',
 'f_9',
 'target'])

## Question 2.6 (20 pts):

We will now use the `to_row` function created above to create a dataframe of the data `df`

In [27]:
df = rdd_complete.map(to_row).toDF()

Explore the dataframe a bit:

```python
df.orderBy('customer_index').show(5)
```

```console
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|customer_index|  f_0|  f_1|  f_2|  f_3|  f_4|  f_5|  f_6|  f_7|  f_8|  f_9| target|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|             1|  0.5|  0.8|-0.49| 0.25| 0.37| 0.73|-0.43| 0.89|-1.85|-0.44|  36.67|
|             3| 0.14|-0.87|-0.94| 0.09|-0.69|-0.29|-0.45| -0.6|-1.28|-0.38|-429.54|
|             4|-0.79|-0.28|-0.26| 0.28|-0.17|-0.51| 0.44|-0.62| 1.82|-0.35| -18.23|
|             5|-0.11| 0.86| -1.3|-0.09|-0.12|-0.47| 0.05| 0.98|-1.59|  0.3|  -5.52|
|             9| 0.57| 0.07|-0.31|-0.92|-0.26| -0.9|-0.06|-0.76| 1.39| 0.01|-111.88|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
only showing top 5 rows
```

In [28]:
# explore it yourself
df.orderBy('customer_index').show(5)

+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|customer_index|  f_0|  f_1|  f_2|  f_3|  f_4|  f_5|  f_6|  f_7|  f_8|  f_9| target|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
|             1|  0.5|  0.8|-0.49| 0.25| 0.37| 0.73|-0.43| 0.89|-1.85|-0.44|  36.67|
|             3| 0.14|-0.87|-0.94| 0.09|-0.69|-0.29|-0.45| -0.6|-1.28|-0.38|-429.54|
|             4|-0.79|-0.28|-0.26| 0.28|-0.17|-0.51| 0.44|-0.62| 1.82|-0.35| -18.23|
|             5|-0.11| 0.86| -1.3|-0.09|-0.12|-0.47| 0.05| 0.98|-1.59|  0.3|  -5.52|
|             9| 0.57| 0.07|-0.31|-0.92|-0.26| -0.9|-0.06|-0.76| 1.39| 0.01|-111.88|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-------+
only showing top 5 rows



The subpackage `pyspark.ml.functions` (aliased as `fn` below) contains many common functions for data analysis. Find the function for computing the __correlation__ (the strength of the linear relationship between two variables) between two columns, the function for computing __absolute__ values, and create a data frame `correlations_df` that contains the following columns in the following order:

1. `c0_target`: correlation between feature 0 and target
1. `c1_target`: correlation between feature 1 and target
1. `c2_target`: correlation between feature 2 and target
1. `c3_target`: correlation between feature 3 and target
1. `c4_target`: correlation between feature 4 and target
1. `c5_target`: correlation between feature 5 and target
1. `c6_target`: correlation between feature 6 and target
1. `c7_target`: correlation between feature 7 and target
1. `c8_target`: correlation between feature 8 and target
1. `c9_target`: correlation between feature 9 and target
1. `sig0`: boolean `true` if the absolute value of the correlation between feature 0 and target is greater than 0.5, `false` o.w.
1. `sig1`: boolean `true` if the absolute value of the correlation between feature 1 and target is greater than 0.5, `false` o.w.
1. `sig2`: boolean `true` if the absolute value of the correlation between feature 2 and target is greater than 0.5, `false` o.w.
1. `sig3`: boolean `true` if the absolute value of the correlation between feature 3 and target is greater than 0.5, `false` o.w.
1. `sig4`: boolean `true` if the absolute value of the correlation between feature 4 and target is greater than 0.5, `false` o.w.
1. `sig5`: boolean `true` if the absolute value of the correlation between feature 5 and target is greater than 0.5, `false` o.w.
1. `sig6`: boolean `true` if the absolute value of the correlation between feature 6 and target is greater than 0.5, `false` o.w.
1. `sig7`: boolean `true` if the absolute value of the correlation between feature 7 and target is greater than 0.5, `false` o.w.
1. `sig8`: boolean `true` if the absolute value of the correlation between feature 8 and target is greater than 0.5, `false` o.w.
1. `sig9`: boolean `true` if the absolute value of the correlation between feature 9 and target is greater than 0.5, `false` o.w.

**Hint: Remember that you can pass a list of columns to `df.select`. You can create such list with list comprehension, saving a lot of code**

In [29]:
# import the package functions as fn
from pyspark.sql import functions as fn

In [30]:
# apply some function to the columns: df.select(...)
df.select('f_7').show()

+-----+
|  f_7|
+-----+
|-0.62|
| -0.8|
|-0.06|
|-1.19|
| 1.05|
|-0.48|
|  0.7|
|-0.85|
|-0.05|
|  0.9|
| 1.05|
| 0.11|
|-0.18|
| 1.07|
| 1.37|
|-0.26|
|-2.44|
|-0.41|
|-0.46|
| 0.58|
+-----+
only showing top 20 rows



In [33]:
# Create the dataframe `correlations_df` here
# YOUR CODE HERE
a= []
b = []

for j in range(10):
    x = df.corr('target','f_'+str(j))
    a.append(x)
    if abs(x) > 0.5:
        b.append(True)
    else:
        b.append(False)

a = a+b

correlations_df = spark.createDataFrame(Row(a))
newColumns = ['c' + str(y) + '_target' for y in range(10)] + ['sig' + str(y) for y in range(10)]
oldColumns = correlations_df.schema.names

for i in range(20):
    correlations_df = correlations_df.withColumnRenamed(oldColumns[i], newColumns[i])
    
correlations_df.show()

+--------------------+-----------------+------------------+--------------------+------------------+--------------------+------------------+-------------------+--------------------+--------------------+-----+----+-----+-----+----+-----+----+-----+-----+-----+
|           c0_target|        c1_target|         c2_target|           c3_target|         c4_target|           c5_target|         c6_target|          c7_target|           c8_target|           c9_target| sig0|sig1| sig2| sig3|sig4| sig5|sig6| sig7| sig8| sig9|
+--------------------+-----------------+------------------+--------------------+------------------+--------------------+------------------+-------------------+--------------------+--------------------+-----+----+-----+-----+----+-----+----+-----+-----+-----+
|0.004550467203235611|0.517541807653123|0.2422254992218166|-0.02725292295686...|0.6109343603342665|-0.01506188705709...|0.5763480950714736|0.06334859042481798|0.007563889344988253|-0.01715842226040008|false|true|false|false

In [32]:
"""20 pts: Check that the dataframe has the correct columns and values. There could be hidden tests!"""
# check column names
column_names = ['c' + str(fi) + '_target' for fi in range(10)] + \
               ['sig' + str(fi) for fi in range(10)]

# column's names and positions in the right order
np.testing.assert_equal(correlations_df.columns, column_names)

from pyspark.sql.types import DoubleType, BooleanType


# the types are correct
np.testing.assert_equal([type(f.dataType) for f in correlations_df.schema.fields], 
                        10*[DoubleType] + 10*[BooleanType])

# the values are correct
np.testing.assert_approx_equal(correlations_df.toPandas().sum().sum(), 
                               4.963039476979365,
                               significant=1)